# Amazon SageMaker - Bring Your Own Container¶
### 学習用インスタンスでの学習
ここでは TensorFlow を使ったサンプルコードを題材に、Amazon SageMaker で独自コンテナを用いた機械学習モデルの学習方法を順を追って説明します。トレーニングスクリプトは既に SageMaker 向けに書き換えられた形で準備され、その上で、独自の学習用コンテナを活用します。 トレーニングスクリプトを SageMaker 向けに書き換える際は [Bring Your Own Model ハンズオンワークショップ](https://github.com/aws-samples/amazon-sagemaker-examples-jp/tree/master/workshop/lab_bring-your-own-model) をご参考に下さい。

また本ハンズオンで活用するデータは、[Bring Your Own Container のためのデータ準備](https://github.com/tkazusa/sagemaker-tensorflow-byoc/blob/master/data_prep.ipynb)をお使い下さい。

In [1]:
import keras

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()

Using TensorFlow backend.


In [2]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install smdebug

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
!docker build -t sm-tf-nightly-gpu container/.

Sending build context to Docker daemon  9.728kB
Step 1/4 : FROM tensorflow/tensorflow:nightly-gpu
 ---> a0490a1fc88d
Step 2/4 : RUN pip install sagemaker-containers
 ---> Using cache
 ---> 62d2077101b5
Step 3/4 : COPY tf_codes /opt/ml/code/
 ---> Using cache
 ---> 96c837471cce
Step 4/4 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> 83e898d67d26
Successfully built 83e898d67d26
Successfully tagged sm-tf-nightly-gpu:latest


## 学習用インスタンスを用いた学習
### ECR への Docker イメージの登録
build した Docker イメージを Docker コンテナレジストリである Amazon ECR へ登録することで、SageMaker の学習時に活用できるようになります。

In [4]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

ecr_repository = 'sm-tf-nightly-gpu'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print('コンテナは {} へ登録されています。'.format(image_uri))

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-1:155580384669:repository/sm-tf-nightly-gpu",
        "registryId": "155580384669",
        "repositoryName": "sm-tf-nightly-gpu",
        "repositoryUri": "155580384669.dkr.ecr.ap-northeast-1.amazonaws.com/sm-tf-nightly-gpu",
        "createdAt": 1597669260.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}
unable to prepare context: unable to evaluate symlinks in Dockerfile path: lstat /home/ec2-user/SageMaker/sagemaker-tensorflow-byoc/Dockerfile: no such file or directory
The push refers to repository [155580384669.dkr.e

030b1dce: Pushing  1.212GB/1.629GBPushing  708.2MB/995.9MB

030b1dce: Pushed   1.637GB/1.629GBlatest: digest: sha256:4feab7ec816db8b8638d1e041f8d0f6357b7c540fa7274df3c5ddc51adb6a0b4 size: 4095
コンテナは 155580384669.dkr.ecr.ap-northeast-1.amazonaws.com/sm-tf-nightly-gpu:latest へ登録されています。


## 実験の作成

In [5]:
import boto3
import time
from smexperiments.experiment import Experiment

boto3_sess = boto3.Session()
sm = boto3_sess.client('sagemaker')

tf_experiment = Experiment.create(
    experiment_name=f"tensorflow-byoc-{int(time.time())}", 
    description="tensorflow experiments", 
    sagemaker_boto_client=sm)

print(tf_experiment)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f5abc8a89b0>,experiment_name='tensorflow-byoc-1597669348',description='tensorflow experiments',tags=None,experiment_arn='arn:aws:sagemaker:ap-northeast-1:155580384669:experiment/tensorflow-byoc-1597669348',response_metadata={'RequestId': 'c95ecfc2-5343-4880-8d47-e09438e3b5f3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c95ecfc2-5343-4880-8d47-e09438e3b5f3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '103', 'date': 'Mon, 17 Aug 2020 13:02:27 GMT'}, 'RetryAttempts': 0})


### GPU を使った学習インスタンスでの学習
登録された Docker イメージと、S3 へアップロードされたデータを用いて学習を行います。
ここでは Optimizer を 'Adam' と 'SGD' とで変更した際の実験管理を行っています。

In [8]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent


role = get_execution_role()
bucket = sagemaker.Session().default_bucket()
output_path = 's3://{}/output'.format(bucket)
data_path = {'train': 's3://{}/data'.format(bucket)}

metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'accuracy: ([0-9\\.]+)'}]

trial_name_map = {}

optimizers = ['adam', 'sgd']

for i, optimizer in enumerate(optimizers):
    # Trial の作成
    trial_name = f'tf-training-job-{optimizer}-{int(time.time())}'
    print(trial_name)
    
    
    tf_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=tf_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    trial_name_map[optimizer] = trial_name
    
    # Estimator の作成とfit。
    hyperparameters = {'batch_size': 64,'epochs': 1, 'optimizer': optimizer}
    estimator = Estimator(image_name=image_uri,
                          role=role,
                          hyperparameters=hyperparameters,
                          train_instance_count=1,
                          train_instance_type='ml.p3.2xlarge',
                          output_path=output_path,
                          metric_definitions=metric_definitions,
                          enable_sagemaker_metrics=True,)
    
    tf_training_job_name = "tf-job-{}".format(int(time.time()))
    
    estimator.fit(data_path,
                 job_name=tf_training_job_name,
                  
                 experiment_config={
                     'TrialName': tf_trial.trial_name,
                     'TrialComponentDisplayName': 'Training',
                 },
                  
                 wait=False)
    
    time.sleep(2)

INFO:sagemaker:Creating training-job with name: tf-job-1597669399


tf-training-job-adam-1597669399


INFO:sagemaker:Creating training-job with name: tf-job-1597669401


tf-training-job-sgd-1597669401


## 学習結果の可視化

In [9]:
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

sess = boto3.Session()

search_expression = {
    "Filters":[
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(sess, sm), 
    experiment_name=tf_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.accuracy.max",
    sort_order="Descending",
    metric_names=['accuracy'],
    parameter_names=['optimizer']
)

analytic_table = trial_component_analytics.dataframe()

In [10]:
analytic_table

,TrialComponentName,DisplayName,SourceArn,optimizer,train - MediaType,train - Value
0,tf-job-1597669401-aws-training-job,Training,arn:aws:sagemaker:ap-northeast-1:155580384669:...,sgd,None,s3://sagemaker-ap-northeast-1-155580384669/data
1,tf-job-1597669386-aws-training-job,Training,arn:aws:sagemaker:ap-northeast-1:155580384669:...,adam,None,s3://sagemaker-ap-northeast-1-155580384669/data
2,tf-job-1597669399-aws-training-job,Training,arn:aws:sagemaker:ap-northeast-1:155580384669:...,adam,None,s3://sagemaker-ap-northeast-1-155580384669/data
